In [1]:
from Fake_news.data import get_data
from Fake_news.encoders import TokenizerTransformer, PadSequencesTransformer
from Fake_news.gcp import storage_upload

import os
import pickle
import joblib
import numpy as np
import pandas as pd
import gensim.downloader as api

from tensorflow import keras
from tensorflow.keras.models import load_model
from sklearn.pipeline import Pipeline
from gensim.models import KeyedVectors
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.preprocessing.sequence import pad_sequences
from memoized_property import memoized_property
# Mlflow wagon server

In [13]:
def get_data(local=False, sample_size=1, nrows=None):
    if local:
        path = '../raw_data/data.csv'
    else:
        path = 'gs://fakenews475/data/data.csv'

    df = pd.read_csv(path, nrows=nrows)
    mask = (df['text'].str.len() < 6000)
    df = df.loc[mask]
    df = df.drop(columns=('Unnamed: 0'))
    df = df.drop(columns=('title'))
    # df_sample = df.sample(frac=sample_size, random_state=3)
    # df_sample.to_csv('df_sample.csv', index=False)
    # df = df_sample.reset_index(drop=True)
    X = df.text
    y = df.label
    return X, y
X,y = get_data(local=True, nrows=100)

In [14]:
X_train, X_val, y_train,y_val = train_test_split(
                 X, y, random_state=3, test_size= 0.1)

In [34]:
def get_pipeline():
    pipeline = joblib.load('../models_LSTM_versions_10.2_pipeline.pkl')
    pipeline.named_steps['model'].model = load_model('../models_LSTM_versions_10.2_lstm.h5')
    return pipeline
pipeline=get_pipeline()

In [37]:
y_val.value_counts()

1    6
0    3
Name: label, dtype: int64

In [42]:
predictions = pipeline.predict(X_val)
pd.DataFrame(predictions).head(20)

,0
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,0


In [17]:
X_val

78    An Embattled Pharmaceutical Company That Sells...
84    Broken2th And Crystal Sun Release “Castaliah”\...
12    Just Shut Up & Play Some Damn Baseball!!\n(Bef...
80    Image copyright Getty Images Image caption Rus...
41    Q. I’m about to purchase an iMac desktop compu...
92    FSU Opponent Preview: Wake Forest\n(Before It'...
98    2017 NLDS: Will Max Scherzer miss Game 1?\n(Be...
59    Lawrence Thomas Officially Makes Switch to Ful...
81    Mr. Affleck has not responded to Ms. McGowan’s...
Name: text, dtype: object

In [30]:
pipeline

Pipeline(memory=None,
     steps=[('tokenizer', TokenizerTransformer()), ('padder', PadSequencesTransformer(maxlen=417)), ('model', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000002123794B348>)])

In [43]:
article = "The UK has become the first country in the world to approve the Pfizer/BioNTech coronavirus vaccine, paving the way for mass vaccination.Britain's medicines regulator, the MHRA, says the jab, which offers up to 95% protection against Covid-19 illness, is safe to be rolled out.The first doses are already on their way to the UK, with 800,000 due in the coming days, Pfizer said.Health Secretary Matt Hancock said the NHS will contact people about jabs.Elderly people in care homes and care home staff have been placed top of the priority list, followed by over-80s and health and care staff.But because hospitals already have the facilities to store the vaccine at -70C, as required, the very first vaccinations are likely to take place there - for care home staff, NHS staff and patients - so none of the vaccine is wasted.A further 648 deaths within 28 days of a positive Covid-19 test were recorded in the UK on Wednesday, with another 16,170 cases reported."

In [44]:
pipeline.predict([article])

array([[0]], dtype=int64)

In [48]:
fake = "In May 1803, as Britain was preparing to end the Treaty of Amiens and declare war on France, a letter was hand delivered to Sir Charles Price, the Lord Mayor of London at the Mansion-house. Allegedly written by Lord Hawkesbury, and sealed with his personal seal, the letter claimed that the dispute with France was amicably settled. The Mayor at once took the letter to the Stock Exchange to share the joyous news."

In [51]:
pipeline.predict([fake])

array([[1]], dtype=int64)

In [52]:
fake2 = "The fake news of an alien attack on America is a classic one. On Sunday, 30 October 1938, the Columbia Broadcasting network aired an adaptation of the 1898 novel War of the Worlds by HG Wells. During the broadcast, the first two thirds of the story were aired over the radio as a series of breaking news alerts and the effect was so realistic that many listeners panicked, believing that there truly was an alien invasion taking place.The show began as an interruption to regular programming announcing that a professor had observed a series of gas explosions on Mars. Later, a bulletin stated that a meteor had fallen in New Jersey, killing 15,000 people. Another news flash contradicted the first, saying that the meteor was actually a cylindrical object containing strange creatures from Mars armed with death ray guns."

In [53]:
pipeline.predict([fake2])

array([[1]], dtype=int64)

In [55]:
real2 = "Stewards on duty on the night of the Manchester bombing did not think Salman Abedi was a threat because they had not been trained properly, a security expert has told the attack inquiry.Col Richard Latham said a managerial fault meant Showsec staff Kyle Lawler and Mohammed Agha failed to respond to suspicious behaviour.They were hardly briefed, he said.The inquiry heard lives could have been saved if security had been quicker to shut the doors of the foyer.Twenty-two people were killed and hundreds more were injured as they left an Ariana Grande concert when Abedi detonated a bomb on 22 May 2017.'Missed opportunities'Col Latham and his colleague Dr David BaMaung said if Mr Lawler and Mr Agha had 'properly communicated' concerns over Abedi there would have been time to close the doors before the concert ended.Dr BaMaung said: 'Irrespective of how quickly we could close the arena doors, realistically if he was going to detonate, people would die that night.'But the potential would be if the procedure had been quickly, the casualties and deaths would have been less.'The inquiry has previously heard how a member of the public reported suspicions about Abedi to Mr Aghar at 22:15 BST.He then alerted Mr Lawler to the report and the pair observed him.Abedi left his position at the back of the City Room, a CCTV 'blind spot', to detonate his device at 22:31."

In [56]:
pipeline.predict([real2])

array([[0]], dtype=int64)